# Rdionica: priprema datoteke `podaci_upitnik.csv` za obradu

In [8]:
options(repr.matrix.max.cols = 120, repr.matrix.max.rows = 500)

U ovom dijelu radionice proći ćemo put od sirovih podataka do podataka na kojima možemo provesti analizu.

Prije nego što se bacimo na učitavanje i proučavanje sirovih podataka, učitat ćemo pakete koje ćemo koristiti.

Pakete učitavamo pozivanjem funkcije `library`, koja kao argument prima ime **jednog** paketa.

In [1]:
# skupina paketa koja sadrži većinu paketa koje
# ćemo koristiti za baratanje podacima
library(tidyverse)

# paket koji sadrži 'pipe' operatore
library(magrittr)

# upozorava na konflikte u imenima funkcija
# koji se javljaju kad više paketa koristi isto
# ime
library(conflicted)

# omogućava učitavanje .SAV fielova
library(foreign)

# paket koji sadrži neke zgodne olakšice
library(wrapr)

# olakšava korištenje relativnih file pathova
library(here)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.7
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract

here() starts at /home/denis/Documents/rdionica


## Učitavanje podatka

Za početak, pogledat ćemo kako izgledaju naši sirovi podaci.

A da bismo to učinili, prvo ih moramo učitati u R.

Pogledat ćemo kako učitati tri vrste datoteka: SPSS-ov `.sav`, Excelov `.xls/xlsx` te generički *comma separated values* file - `.csv`.

### SPSS - .sav

### Excel - .xls(x)

### Comma separated values - .csv

*Comma separeted value* datoteke su točno to što ime kaže - podaci koji su strukturirani kao vrijednosti odvojene zarezima, gdje se svaki unos (na primjer sudionik) nalazi u zasebnom redu, a vrijednosti varijabli koje su uz njega povezane ispisane su redom i odvojene su zarezima.

U prvom redu (koji funkcije u R-u često nazivaju **header**) obično se nalaze imena varijabli, a u ostalim redovima su njihove vrijednosti.

Ovako izgledaju prva dva reda i prvih nekoliko stupaca datoteke `podaci_upitnik.csv`:

```
attitudesAndNorms01,attitudesAndNorms02,attitudesAndNorms03, ...
5,5,5,5,4, ...
```

Podatke u `.csv` formatu možemo učitati pomoću funkcije `read_csv` iz `readr` paketa (koji je automatski učitan kad smo učitali `tidyverse`). Osnovni (base) R ima funkciju `read.csv` koja obavlja isti zadatak, ali neki R developeri preporučuju korištenje `read_csv` funkcije (na primjer, Hadley Wickham i Garret Grolemund: http://r4ds.had.co.nz/import.html).

U skladu s tom preporukom, koristit ćemo `read_csv`. Podatke iz datoteke `podaci_upitnik.csv` možemo učitati ovako:

In [2]:
podaci <- read_csv(file = here('podaci', 'podaci_upitnik.csv'))

Parsed with column specification:
cols(
  .default = col_integer(),
  pi_education = col_character(),
  pi_gender = col_character(),
  pi_ideology = col_character(),
  pi_income = col_character(),
  pi_nationality = col_character(),
  pi_previousDonations = col_character()
)
See spec(...) for full column specifications.


Poruka koju dobivamo obavještava nas o tome kako su određene varijable reprezentirane. Vidimo da su varijable koje počinju s `pi` reprezentirane kao `character`. Ako pozovemo funkciju `spec`, vidjet ćemo specifikacije svih varijabli.

Budući da pozivanjem funkcije `str` zapravo dobivamo manje-više iste podatke, pozvat ćemo samo nju. Njen output pomoći će nam da vidimo jesu li podaci reprezentirani onako kako bismo očekivali.

In [ ]:
str(podaci)

Funkcije `head` i `tail` omogućuju nam da na brzinu vidimo (po defaultu) prvih 6 odnosno posljednjih 6 redova tablice.

In [ ]:
head(podaci)

In [ ]:
tail(podaci, 3)

Ove funkcije pomažu nam pri pregledavanju strukture podataka i njihovih sirovih vrijednosti.

Osnovnu deskriptivnu statistiku možemo dobiti pomoću generičke funkcije `summary`. Output funkcije ovisi o tipu objekta koji u nju stavimo. Ako joj damo `data.frame`, dobit ćemo grubu deskriptivnu statistiku, ovisno o tipu vrijednosti pojedinih varijabli.

Dobro je znati i za funkciju `describe` iz paketa `psych`, koja daje dosta detaljniju deskriptivnu statistiku numeričkih varijabli.

Idemo vidjeti output tih dviju funkcija kad u nju stavimo neke numeričke i neke kategorijalne (`factor`) stupce iz našeg `data.framea` `podaci`.

In [3]:
summary(podaci[, wrapr::qc(attitudesAndNorms01, pi_education, pi_gender)])

 attitudesAndNorms01 pi_education        pi_gender        
 Min.   :2.00        Length:100         Length:100        
 1st Qu.:4.00        Class :character   Class :character  
 Median :5.00        Mode  :character   Mode  :character  
 Mean   :5.04                                             
 3rd Qu.:6.00                                             
 Max.   :8.00                                             

Vidimo tri stvari: (1) `summary` nije pretjerano koristan za varijable koje su tipa `character` i (2-3) pojavili su se nova sintaksa i nova funkcija.

`qc` je funkcija iz paketa `wrapr` koja nas oslobađa pisanja navodnika pri korištenju funkcije `c`. `qc` je, dakle, *quoted combine*.

Korištenjem `::` sintakse označili smo da je funkcija `qc` iz paketa `wrapr`. Pri pozivanju funkcija iz drugih paketa **nije nužno** pisati `::`; to smo vidjeli i kod pozivanja funkcije `read_csv` iz paketa `readr`.

Ipak, važno je znati tu sintaksu iz dva razloga.

Prvo, korištenjem `::` možemo pozvati funkciju iz paketa koji prethodno nismo učitali (što ćemo vidjeti pri pozivanju funkcije `describe`.

Drugo, u slučaju da dva paketa imaju funkcije koje se jednako zovu, `::` nam omogućava da specificiramo koju funkciju želimo pozvati. Budući da smo učitali paket `conflicted`, R će nas upozoriti ako dođe do konflikta te nas tražiti da specificiramo koju funkciju hoćemo pozvati, koristeći `::`.

Sad ćemo pogledati output funkcije `describe` na tim istim varijablama.

Nastavit ću koristiti `::` notaciju tako da bude jasno iz kojeg paketa dolazi koja funkcija (osim ako je spomenuto u tekstu).

**BAM-BAM PIPE**

In [7]:
podaci %>%
select(attitudesAndNorms01, pi_education, pi_gender) %>% psych::describe(.)

Warning message in psych::describe(.):
“NAs introduced by coercion”Warning message in psych::describe(.):
“NAs introduced by coercion”Warning message in FUN(newX[, i], ...):
“no non-missing arguments to min; returning Inf”Warning message in FUN(newX[, i], ...):
“no non-missing arguments to min; returning Inf”Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”Warning message in FUN(newX[, i], ...):
“no non-missing arguments to max; returning -Inf”

,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
attitudesAndNorms01,1,100,5.04,1.427861,5,5.0625,1.4826,2,8,6,-0.1931472,-0.5916736,0.1427861
pi_education*,2,100,NaN,NA,NA,NaN,NA,Inf,-Inf,-Inf,NA,NA,NA
pi_gender*,3,100,NaN,NA,NA,NaN,NA,Inf,-Inf,-Inf,NA,NA,NA


I dalje nije korisno za `character` varijable, ali omogućava digresiju u svijet pipeova.

## Pipe

Pipe su posebni operatori iz `magrittr` paketa. One omogućavaju kraće i, često, razumljivije pisanje koda.

Pipa uzima output izraza sa svoje lijeve strane i daje ga kao argument funkciji na svojoj desnoj strani.

Osnovna pipa je `%>%`. Ona se nalazi i u paketu `dplyr` (koji se učitava kad učitamo `tidyverse`) i u paketu `magrittr`. Posebno smo učitali `magrittr` jer s njim dolaze i neke pipe kojih nema u `dplyruu`.

Sad ćemo proći kroz pipe koje `magrittr` nudi.

### %>%

## Reference

Grolemund, G. i Wickham, H. *R for data science*. O'Reilly Media, Inc.

Pipe: https://cran.r-project.org/web/packages/magrittr/vignettes/magrittr.html

## Epilog

In [ ]:
sessionInfo() 